# ======test set에 대한 검증=======

In [ ]:
# 저장된 모델의 경로
saved_model_path = "best_model"  # .h5 확장자 제거

# 저장된 모델 불러오기
loaded_model = tf.keras.models.load_model(saved_model_path)

In [ ]:
# test_essays 데이터의 토큰화 진행 상황을 표시
test_input_ids = []
test_attention_masks = []

for text in tqdm(test_essays['text'].tolist(), desc="토큰화 진행중"):
    encoded = tokenizer.encode_plus(text, 
                                    add_special_tokens=True, 
                                    max_length=256, 
                                    padding='max_length', 
                                    truncation=True, 
                                    return_attention_mask=True, 
                                    return_tensors='np')
    test_input_ids.append(encoded['input_ids'][0])
    test_attention_masks.append(encoded['attention_mask'][0])



test_inputs = {
    "input_ids": np.array(test_input_ids),
    "attention_mask": np.array(test_attention_masks)
}  
print("test_inputs shape:", test_inputs["input_ids"].shape)

In [ ]:
# 검증 데이터에 대한 예측 로짓을 얻음
prediction_logits = loaded_model.predict(test_inputs).logits

# 예측 결과를 적절한 형태로 변환
prediction_probs = tf.nn.softmax(prediction_logits, axis=1)[:, 1].numpy()

# 결과를 캐글 제출 형식에 맞게 데이터프레임으로 변환
submission = pd.DataFrame({
    'id': test_essays['id'],  # 샘플 ID
    'prediction': prediction_probs  # 예측 확률
})

# 결과를 CSV 파일로 저장
submission.to_csv('submission.csv', index=False)